In [1]:
import torch
import jax
import jax.numpy as jnp
from jax import random
from flax.serialization import from_bytes
import sys
sys.path.append('../../')
from resnet20_jax import BLOCKS_PER_GROUP, ResNet
from flax.traverse_util import flatten_dict
import numpy as np
from utils import flatten_params, unflatten_params
from weight_matching import resnet20_permutation_spec
from resnet20_torch import resnet20

/srv/share/gstoica3/miniconda3/envs/open-mmlab/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = ResNet(blocks_per_group=BLOCKS_PER_GROUP["resnet20"],
                   num_classes=512,
                   width_multiplier=4)

In [3]:
key1 , key2 = random.split(random.PRNGKey(0))
model_params = model.init(key2, jnp.zeros((1, 32, 32, 3)))
# from_bytes(model.init(random.PRNGKey(0), jnp.zeros((1, 32, 32, 3)))["params"], fh.read())

2023-03-03 14:33:27.122840: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-03 14:33:27.122904: W external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
model_params

FrozenDict({
    params: {
        conv1: {
            kernel: DeviceArray([[[[-0.07793207,  0.26039168,  0.28132746, ...,
                            -0.28206113, -0.25232807, -0.37629896],
                           [-0.03069908,  0.34608918, -0.2144393 , ...,
                             0.3385243 ,  0.14656727, -0.03853216],
                           [-0.25068933,  0.19329914, -0.08257911, ...,
                             0.27509916,  0.06269549,  0.13682872]],
            
                          [[ 0.06289499, -0.18571858,  0.2057212 , ...,
                            -0.05704458, -0.07370687,  0.169397  ],
                           [ 0.00085753, -0.20943874, -0.35513303, ...,
                            -0.10990165,  0.07941165,  0.04222045],
                           [ 0.07988793, -0.02642079,  0.05129561, ...,
                            -0.02099439,  0.25206962,  0.37053823]],
            
                          [[-0.00523731, -0.00919603,  0.11126664, ...,
        

In [5]:

# def load_model(filepath):
#     with open(filepath, "rb") as fh:
#         return from_bytes(
#             model.init(random.PRNGKey(0), jnp.zeros((1, 32, 32, 3)))["params"], fh.read())
# model = load_model("../v7")
params = flatten_params(model_params)

In [6]:
params

{'params/blockgroups_0/blocks_0/conv1/kernel': DeviceArray([[[[-0.04106247,  0.00311194,  0.05538438, ...,
                  0.04658882,  0.00372277, -0.02551748],
                [ 0.00520444,  0.05231606, -0.02031168, ...,
                  0.00340005,  0.00819752, -0.08711487],
                [-0.03161848,  0.04728803, -0.0198846 , ...,
                 -0.05147438,  0.02048825, -0.0719903 ],
                ...,
                [ 0.04043911,  0.05911505,  0.03838796, ...,
                 -0.01465605, -0.02082756, -0.04794794],
                [ 0.00462797,  0.00936126, -0.05212557, ...,
                  0.04387324,  0.05325392,  0.06132902],
                [ 0.02766319, -0.03328244, -0.02280221, ...,
                  0.00419793, -0.03005159,  0.04268934]],
 
               [[ 0.01134065, -0.01626284,  0.00836775, ...,
                  0.02975465,  0.05971113,  0.01153979],
                [-0.03732794, -0.08952981,  0.07854121, ...,
                 -0.07682095,  0.00383221, 

In [7]:
spec = resnet20_permutation_spec().axes_to_perm
# this lines up the keys of corresponding dictionaries, at least it seems.

In [8]:
def find_pairs(str_splits):
    pairs = []
    for i, str_split_i in enumerate(str_splits):
        split_i = set([int(k) for k in str_split_i.split('_')])
        for str_split_j in str_splits[i+1:]:
            split_j = set([int(k) for k in str_split_j.split('_')])
            if len(split_i.intersection(split_j)) == 0:
                pairs.append((str_split_i, str_split_j))
    return pairs


def split_str_to_ints(split):
    return [int(i) for i in split.split('_')]


def is_valid_pair(model_dir, pair, model_type):
    paths = os.listdir(os.path.join(model_dir, pair[0]))
    for path in paths:
        if model_type in path:
            return True
    return False

In [19]:
import os

In [20]:
# model1_save_path = "/srv/share/jbjorner3/checkpoints/REPAIR/resnet20x4_CIFAR50_clses[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49].pth.tar"
# model2_save_path = "/srv/share/jbjorner3/checkpoints/REPAIR/resnet20x4_CIFAR50_clses[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99].pth.tar"
# sd = torch.load('model_1.th')["state_dict"]
# sd = {k: v.cpu() for k, v in sd.items()}
# len(sd.keys())
DEVICE = "cpu"

model_dir = '/srv/share2/gstoica3/checkpoints/cifar50_traincliphead/'
model_name = 'resnet20x4'
eval_pair = 0
pair = [pair for pair in find_pairs(os.listdir(model_dir)) if is_valid_pair(model_dir, pair, model_name)][eval_pair]
model_save_paths = [os.path.join(model_dir, split, f'{model_name}_v0.pth.tar') for split in pair]


model1 = resnet20(w=4).to(DEVICE)
sd = torch.load(model_save_paths[0], map_location=torch.device(DEVICE))
sd = {k: v.cpu() for k, v in sd.items()}
model1.load_state_dict(sd)

<All keys matched successfully>

In [21]:
list(sd.keys()).__len__()

128

In [22]:
import pdb

In [23]:
def torch_to_linen(torch_params, get_flax_keys):
    """Convert PyTorch parameters to Linen nested dictionaries"""

    def add_to_params(params_dict, nested_keys, param, is_conv=False):
        if len(nested_keys) == 1:
            key, = nested_keys
            try:
                params_dict[key] = np.transpose(param, (2, 3, 1, 0)) if is_conv else np.transpose(param)
            except:
                pdb.set_trace()
        else:
            assert len(nested_keys) > 1
            first_key = nested_keys[0]
            if first_key not in params_dict:
                params_dict[first_key] = {}
            add_to_params(params_dict[first_key], nested_keys[1:], param, ('conv' in first_key and \
                                                                         nested_keys[-1] != 'bias'))

    flax_params = {'params': {}, 'batch_stats': {}}
    for key, tensor in torch_params.items():
        flax_keys = get_flax_keys(key.split('.'))
        if flax_keys[-1] is not None:
            if flax_keys[-1] in ('mean', 'var'):
                add_to_params(flax_params['batch_stats'], flax_keys, tensor.detach().numpy())
            else:
                add_to_params(flax_params['params'], flax_keys, tensor.detach().numpy())

    return flax_params

In [24]:
list((v.shape, k) for k, v in sd.items())

[(torch.Size([64, 3, 3, 3]), 'conv1.weight'),
 (torch.Size([64]), 'bn1.weight'),
 (torch.Size([64]), 'bn1.bias'),
 (torch.Size([64]), 'bn1.running_mean'),
 (torch.Size([64]), 'bn1.running_var'),
 (torch.Size([]), 'bn1.num_batches_tracked'),
 (torch.Size([64, 64, 3, 3]), 'layer1.0.conv1.weight'),
 (torch.Size([64]), 'layer1.0.bn1.weight'),
 (torch.Size([64]), 'layer1.0.bn1.bias'),
 (torch.Size([64]), 'layer1.0.bn1.running_mean'),
 (torch.Size([64]), 'layer1.0.bn1.running_var'),
 (torch.Size([]), 'layer1.0.bn1.num_batches_tracked'),
 (torch.Size([64, 64, 3, 3]), 'layer1.0.conv2.weight'),
 (torch.Size([64]), 'layer1.0.bn2.weight'),
 (torch.Size([64]), 'layer1.0.bn2.bias'),
 (torch.Size([64]), 'layer1.0.bn2.running_mean'),
 (torch.Size([64]), 'layer1.0.bn2.running_var'),
 (torch.Size([]), 'layer1.0.bn2.num_batches_tracked'),
 (torch.Size([64, 64, 3, 3]), 'layer1.1.conv1.weight'),
 (torch.Size([64]), 'layer1.1.bn1.weight'),
 (torch.Size([64]), 'layer1.1.bn1.bias'),
 (torch.Size([64]), 'laye

In [25]:
len(sd), len(spec)

(128, 107)

In [26]:
dict(zip(sd.keys(), spec.keys()))

{'conv1.weight': 'conv1/kernel',
 'bn1.weight': 'norm1/scale',
 'bn1.bias': 'norm1/bias',
 'bn1.running_mean': 'norm1/mean',
 'bn1.running_var': 'norm1/var',
 'bn1.num_batches_tracked': 'blockgroups_0/blocks_0/conv1/kernel',
 'layer1.0.conv1.weight': 'blockgroups_0/blocks_0/norm1/scale',
 'layer1.0.bn1.weight': 'blockgroups_0/blocks_0/norm1/bias',
 'layer1.0.bn1.bias': 'blockgroups_0/blocks_0/norm1/mean',
 'layer1.0.bn1.running_mean': 'blockgroups_0/blocks_0/norm1/var',
 'layer1.0.bn1.running_var': 'blockgroups_0/blocks_0/conv2/kernel',
 'layer1.0.bn1.num_batches_tracked': 'blockgroups_0/blocks_0/norm2/scale',
 'layer1.0.conv2.weight': 'blockgroups_0/blocks_0/norm2/bias',
 'layer1.0.bn2.weight': 'blockgroups_0/blocks_0/norm2/mean',
 'layer1.0.bn2.bias': 'blockgroups_0/blocks_0/norm2/var',
 'layer1.0.bn2.running_mean': 'blockgroups_0/blocks_1/conv1/kernel',
 'layer1.0.bn2.running_var': 'blockgroups_0/blocks_1/norm1/scale',
 'layer1.0.bn2.num_batches_tracked': 'blockgroups_0/blocks_1/nor

In [29]:
from jax import random as jax_random
import jax
import re

In [30]:
# model1_save_path = "/srv/share/jbjorner3/checkpoints/REPAIR/resnet20x4_CIFAR50_clses[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49].pth.tar"
# model2_save_path = "/srv/share/jbjorner3/checkpoints/REPAIR/resnet20x4_CIFAR50_clses[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99].pth.tar"
# sd = torch.load('model_1.th')["state_dict"]
# sd = {k: v.cpu() for k, v in sd.items()}
# len(sd.keys())

model2 = resnet20(w=4).to(DEVICE)
sd = torch.load(model_save_paths[1], map_location=torch.device(DEVICE))
sd = {k: v.cpu() for k, v in sd.items()}
model2.load_state_dict(sd)

<All keys matched successfully>

In [31]:
# init torch_resnet20
# print(torch_model)
# print(dict(torch_model.state_dict()))
torch_dict_sizes = jax.tree_util.tree_map(lambda x: list(x.shape), dict(model2.state_dict()))
def not_bn_stat(x):
    return "num_batches_tracked" not in x and 'mean' not in x and 'var' not in x
torch_list_sizes = list(item for item in torch_dict_sizes.items() if not not_bn_stat(item[0]) and "num_batches" not in item[0])
torch_list_sizes = list(item for item in torch_dict_sizes.items() if not_bn_stat(item[0]))
print(torch_list_sizes.__len__())
torch_list_sizes

65


[('bn1.bias', [64]),
 ('bn1.weight', [64]),
 ('conv1.weight', [64, 3, 3, 3]),
 ('layer1.0.bn1.bias', [64]),
 ('layer1.0.bn1.weight', [64]),
 ('layer1.0.bn2.bias', [64]),
 ('layer1.0.bn2.weight', [64]),
 ('layer1.0.conv1.weight', [64, 64, 3, 3]),
 ('layer1.0.conv2.weight', [64, 64, 3, 3]),
 ('layer1.1.bn1.bias', [64]),
 ('layer1.1.bn1.weight', [64]),
 ('layer1.1.bn2.bias', [64]),
 ('layer1.1.bn2.weight', [64]),
 ('layer1.1.conv1.weight', [64, 64, 3, 3]),
 ('layer1.1.conv2.weight', [64, 64, 3, 3]),
 ('layer1.2.bn1.bias', [64]),
 ('layer1.2.bn1.weight', [64]),
 ('layer1.2.bn2.bias', [64]),
 ('layer1.2.bn2.weight', [64]),
 ('layer1.2.conv1.weight', [64, 64, 3, 3]),
 ('layer1.2.conv2.weight', [64, 64, 3, 3]),
 ('layer2.0.bn1.bias', [128]),
 ('layer2.0.bn1.weight', [128]),
 ('layer2.0.bn2.bias', [128]),
 ('layer2.0.bn2.weight', [128]),
 ('layer2.0.conv1.weight', [128, 64, 3, 3]),
 ('layer2.0.conv2.weight', [128, 128, 3, 3]),
 ('layer2.0.shortcut.0.weight', [128, 64, 3, 3]),
 ('layer2.0.short

In [32]:
model1_state_dict = dict(model1.state_dict())
model2_state_dict = dict(model2.state_dict())

In [16]:
def fix_keys(old_key):
    new_key = old_key
    substitutions =[
        ("bn", "norm"),
        ("layer", "blockgroups_"),
        ("running_mean", "mean"),
        ("running_var", "var"),
        ("weight", "kernel")
    ]
    for sub in substitutions:
        new_key = new_key.replace(sub[0], sub[1])
    new_key = re.sub(r"blockgroups_(\d)", lambda x: f"blockgroups_{int(x.group(1))-1}", new_key)
    new_key = re.sub(r"blockgroups_(\d)\.(\d)\.", "blockgroups_\g<1>.blocks_\g<2>.", new_key)
    new_key = re.sub(r"shortcut\.", "shortcut.layers_", new_key)
    new_key = re.sub(r"norm(\d).kernel", "norm\g<1>.scale", new_key)
    return new_key
def expand_dict(parent_dict, key, value):
    # expand dict along periods of the key
    keys = key.split(".")
    if 'shortcut' in keys and 'layers_1' in keys and 'kernel' in keys:
        keys[-1] = 'scale'
    if 'linear' in keys:
        keys[keys.index('linear')] = 'dense'
    curr_dict = parent_dict
    for new_key in keys[:-1]:
        if curr_dict.get(new_key, None) == None:
            curr_dict[new_key] = dict()
        curr_dict = curr_dict[new_key]    
    
    curr_dict[keys[-1]] = value

def fix_vals(old_key, old_val):

    new_val = old_val.detach().cpu().numpy()
    if "conv" in old_key or 'shortcut.0' in old_key:
        new_val = jnp.transpose(new_val, (2, 3, 1, 0))
    elif 'linear.weight' in old_key:
        new_val = jnp.transpose(new_val, (1, 0))
    return new_val

def convert_torch_sd_to_flax_sd(torch_state_dict):
    torch_state_dict = {fix_keys(k): fix_vals(k, v) for k, v in torch_state_dict.items()}

    torch_batch_stats = {k: v for k, v in torch_state_dict.items() if not not_bn_stat(k) and "num_batches" not in k}
    torch_params = {k: v for k, v in torch_state_dict.items() if not_bn_stat(k)}
    torch_batch_stats_expanded = dict()
    torch_params_expanded = dict()
    for k, v in torch_batch_stats.items():
        expand_dict(torch_batch_stats_expanded, k, v)
    for k, v in torch_params.items():
        expand_dict(torch_params_expanded, k, v)
    flax_dict = {
            "batch_stats": torch_batch_stats_expanded, 
            "params": torch_params_expanded
        }
    return flax_dict

In [33]:
model1_flax_sd = convert_torch_sd_to_flax_sd(model1_state_dict)
model2_flax_sd = convert_torch_sd_to_flax_sd(model2_state_dict)

In [34]:
from pprint import pprint
pprint(jax.tree_util.tree_map(lambda x: x.shape, model1_flax_sd))

{'batch_stats': {'blockgroups_0': {'blocks_0': {'norm1': {'mean': (64,),
                                                          'var': (64,)},
                                                'norm2': {'mean': (64,),
                                                          'var': (64,)}},
                                   'blocks_1': {'norm1': {'mean': (64,),
                                                          'var': (64,)},
                                                'norm2': {'mean': (64,),
                                                          'var': (64,)}},
                                   'blocks_2': {'norm1': {'mean': (64,),
                                                          'var': (64,)},
                                                'norm2': {'mean': (64,),
                                                          'var': (64,)}}},
                 'blockgroups_1': {'blocks_0': {'norm1': {'mean': (128,),
                                              

In [35]:
pprint(jax.tree_util.tree_map(lambda x: x.shape, model_params))

FrozenDict({
    params: {
        blockgroups_0: {
            blocks_0: {
                conv1: {
                    kernel: (3, 3, 64, 64),
                },
                conv2: {
                    kernel: (3, 3, 64, 64),
                },
                norm1: {
                    bias: (64,),
                    scale: (64,),
                },
                norm2: {
                    bias: (64,),
                    scale: (64,),
                },
            },
            blocks_1: {
                conv1: {
                    kernel: (3, 3, 64, 64),
                },
                conv2: {
                    kernel: (3, 3, 64, 64),
                },
                norm1: {
                    bias: (64,),
                    scale: (64,),
                },
                norm2: {
                    bias: (64,),
                    scale: (64,),
                },
            },
            blocks_2: {
                conv1: {
                    kerne

FrozenDict({
    params: {
        conv1: {
            kernel: DeviceArray([[[[-0.07793207,  0.26039168,  0.28132746, ...,
                            -0.28206113, -0.25232807, -0.37629896],
                           [-0.03069908,  0.34608918, -0.2144393 , ...,
                             0.3385243 ,  0.14656727, -0.03853216],
                           [-0.25068933,  0.19329914, -0.08257911, ...,
                             0.27509916,  0.06269549,  0.13682872]],
            
                          [[ 0.06289499, -0.18571858,  0.2057212 , ...,
                            -0.05704458, -0.07370687,  0.169397  ],
                           [ 0.00085753, -0.20943874, -0.35513303, ...,
                            -0.10990165,  0.07941165,  0.04222045],
                           [ 0.07988793, -0.02642079,  0.05129561, ...,
                            -0.02099439,  0.25206962,  0.37053823]],
            
                          [[-0.00523731, -0.00919603,  0.11126664, ...,
        

In [40]:
# translate the shapes of the tensors to match with jax's formatting: https://flax.readthedocs.io/en/latest/advanced_topics/convert_pytorch_to_flax.html
model = ResNet(blocks_per_group=BLOCKS_PER_GROUP["resnet20"],
                   num_classes=512,
                   width_multiplier=4)

x_key = random.PRNGKey(0)
# jnp.zeros((1, 32, 32, 3))
x = random.normal(x_key, (1, 32, 32, 3))
out = model.apply(model1_flax_sd, x)
out.shape

(1, 512)

In [42]:
import pickle
import os

flax_model_save_paths = [os.path.join(model_dir, split, f'{model_name}_flax_v0.pkl') for split in pair]

pickle.dump(
    model1_flax_sd, 
    open(
        os.path.join(model_dir, pair[0], f'{model_name}_flax_v0.pkl'), 'wb'
        )
    )
pickle.dump(
    model2_flax_sd, 
    open(
        os.path.join(model_dir, pair[1], f'{model_name}_flax_v0.pkl'), 'wb'
        )
    )

# Convert Back to Pytorch

In [44]:
def load_pickle(path):
    return pickle.load(open(path, 'rb'))



rebasin_path = '/srv/share2/gstoica3/checkpoints/cifar50_traincliphead/4_6_7_8_10_11_13_14_17_20_26_27_28_31_33_36_37_38_39_40_43_44_46_48_51_52_53_55_56_57_59_60_62_63_64_66_70_71_72_75_76_78_80_84_85_89_90_92_98_99/flax_cifar50_2_permuted_to_1.pkl'
jax_params_dict = load_pickle(rebasin_path)


In [56]:
# convert back to pytorch from jax.
def collapse_dict(jax_params_dict):
    torch_params_dict = dict()
    
    for old_dict in [jax_params_dict]:
        # print(old_dict)
        recursively_build_dict([], old_dict, torch_params_dict)
    # pprint(torch_params_dict)
    torch_params_dict = {fix_keys(k): v for k, v in torch_params_dict.items()}
    torch_params_dict = {k: fix_vals(k, v) for k, v in torch_params_dict.items()}

    return torch_params_dict
def recursively_build_dict(old_keys, old_dict, new_dict):
    if isinstance(old_dict, flax.core.frozen_dict.FrozenDict):
        for old_key, old_val in old_dict.items():
            recursively_build_dict(old_keys + [old_key], old_val, new_dict)
    else:
        # now we have an array to convert
        new_dict[".".join(old_keys)] = old_dict
def fix_keys(old_key):
    new_key = old_key
    new_key = re.sub(r"norm(\d).scale", "norm\g<1>.kernel", new_key)
    new_key = re.sub(r"shortcut\.layers_", "shortcut.", new_key)
    new_key = re.sub(r"shortcut\.1\.scale", "shortcut.1.weight", new_key)
    new_key = re.sub(r"blockgroups_(\d)\.blocks_(\d)\.", "blockgroups_\g<1>.\g<2>.", new_key)
    new_key = re.sub(r"blockgroups_(\d)", lambda x: f"blockgroups_{int(x.group(1))+1}", new_key)
    substitutions =[("bn", "norm"),("layer", "blockgroups_"),("running_mean", "mean"),("running_var", "var"),("weight", "kernel"),("linear","dense")]
    for sub in substitutions[::-1]:
        new_key = new_key.replace(sub[1], sub[0]) # in reverse order of old fix_keys
    return new_key
def fix_vals(old_key, old_val):
    new_val = old_val
    if "conv" in old_key or 'shortcut.0' in old_key:
        # new_val = jnp.transpose(new_val, (2, 3, 1, 0))
        new_val = jnp.transpose(new_val, (3, 2, 0, 1))
    elif 'linear.weight' in old_key:
        new_val = jnp.transpose(new_val, (1, 0))

    new_val = torch.tensor(np.array(new_val))
    
    return new_val

# def find_permutation(bn_bias_permuted, bn_bias_not_permuted):
#     # find the picking order necessary to index the not permuted to achieve the permuted.
#     abs_dif_matrix = np.abs(bn_bias_permuted[None, :] - bn_bias_not_permuted[:, None])
#     return np.argmin(abs_dif_matrix, axis=0)
# def fix_batch_stats_permutations(new_torch_params_dict, old_torch_params_dict):
#     # correct the batch stats based on the permutations in the weight and bias (should be the same?)
#     def fix_batch_stats_val(k, v):
#         if "bn" in k or "shortcut.1" in k:
#             print(k)
#             if "running_mean" in k or "running_var" in k:
#                 bn_mean_key = k.replace("running_mean", "bias").replace("running_var", "bias")
#                 perm_bn_mean = new_torch_params_dict[bn_mean_key]
#                 not_perm_bn_mean = old_torch_params_dict[bn_mean_key]
#                 perm_indexer = find_permutation(perm_bn_mean, not_perm_bn_mean)
#                 return
#             # remove the word bias or weight, and replace with running mean and running var to get access to these
#         return v
#     fixed_torch_params_dict = {k: v for k, v in new_torch_params_dict.items()}


# to test set equality both ways.
# pprint({item[0] for item in torch_list_sizes}.difference(output_dict.keys()))
# pprint(set(output_dict.keys()).difference({item[0] for item in torch_list_sizes}))
# pprint(jax.tree_util.tree_map(lambda x: x.shape, output_dict))
# pprint()
# correct_sizes_dict = {k:torch.Size(v) for k, v in torch_list_sizes}
# for key, val in output_dict.items():
#     if correct_sizes_dict[key] != val.shape:
#         print("incorrect matching", key, val.shape, correct_sizes_dict[key])
#         print(correct_sizes_dict[key])
#         print(val.shape)
#         print(fix_vals(key, val.detach().numpy()).shape)
# find and apply batch stats permutation based on the permutation of the bn bias.
# jax_params.keys()
# torch_list_sizes
# old_torch_params_dict = dict(torch_model.state_dict())

In [47]:
import flax

In [54]:
flax.core.frozen_dict.FrozenDict(jax_params_dict)

flax.core.frozen_dict.FrozenDict

In [57]:
output_dict = collapse_dict(flax.core.frozen_dict.FrozenDict(jax_params_dict))
print(len(output_dict))

save_path = '/srv/share2/gstoica3/checkpoints/cifar50_traincliphead/4_6_7_8_10_11_13_14_17_20_26_27_28_31_33_36_37_38_39_40_43_44_46_48_51_52_53_55_56_57_59_60_62_63_64_66_70_71_72_75_76_78_80_84_85_89_90_92_98_99/rebasin_cifar50_2_permuted_to_1.pkl'
pickle.dump(output_dict, open(save_path, "wb"))

107
